In this project we analyze our data to match reported chargebacks (excel file) with transactions from the database. First we put an excel file to SQLite database and read it by using Pandas dataframes. Then we create an engine to connect with our database and load chose data into pandas dataframes. We select and filter data from database and match them to data comes from excel file.

In [2]:
import sqlite3
import pandas as pd

Read report:

In [3]:
df1 = pd.read_excel('Processing Report.xlsx')
df1.head()

,Transaction Datetime,Currnecy,Amount,Masked CCN,Card Brans,ARN
0,2017-10-02 01:19:58,USD,70,483316******3047,visa,2DD77FW7-1506899998-100375982-D4QGK72CC0
1,2017-10-02 08:21:16,EUR,100,403905******6257,visa,2DD77FW7-1506925276-100375856-3CLLAK8JZ0
2,2017-10-02 16:57:29,USD,50,485340******8306,visa,2DD77FW7-1506956249-100632165-A35L6E461Z
3,2017-10-03 06:14:16,EUR,240,494172******3012,visa,2DD77FW7-1507004056-100632165-HZCR6EGFSX
4,2017-10-03 09:44:27,EUR,120,443047******8271,visa,2DD77FW7-1507016667-100375856-WEYWF1CIRI


In [4]:
#shape of data
df1.shape

(193, 6)

Connecting with database:

In [5]:
conn = sqlite3.connect("db.sqlite")

In [6]:
cur = conn.cursor()

Read data from our database:

In [7]:
df2 = pd.read_sql('SELECT * FROM Cards', conn)
df2.head()

,id,created_at,bin,last_4,exp_year,exp_month,brand,bin_country
0,1492866731,2017-04-22 15:12:11,533248,1186,2019,12,mastercard,CZ
1,1508145460,2017-10-16 11:17:40,422676,1976,2023,4,visa,VE
2,1488642434,2017-03-04 16:47:14,451769,4446,2018,6,visa,MX
3,1498663410,2017-06-28 17:23:30,450644,1644,2020,2,visa,HR
4,1502538662,2017-08-12 13:51:02,403995,8031,2024,7,visa,FR


In [8]:
df2.shape

(9997, 8)

In [9]:
df3 = pd.read_sql('SELECT * FROM Customers', conn)
df3.head()

,id,created_at,card_id,name,email,ip_addr,ip_country
0,1492866732,2017-04-22 15:12:12,1492866731,Jasmine Parks,jasmine-parks@hotmail.com,73.190.95.73,CZ
1,1508145461,2017-10-16 11:17:41,1508145460,Dusty Ryan,dusty-ryan@yahoo.com,71.76.39.197,VE
2,1488642435,2017-03-04 16:47:15,1488642434,Ila Rogers,rogers.ila@hotmail.com,71.63.38.131,MX
3,1498663411,2017-06-28 17:23:31,1498663410,Victoria Graham,victoria.graha@hotmail.com,97.32.2.154,HR
4,1502538663,2017-08-12 13:51:03,1502538662,Alisa Roland,roland.alisa7789@aol.com,2600:1700:b720:25a0:5548:b48a:e002:3af3,FR


In [10]:
df3.shape

(9997, 7)

In [11]:
df4 = pd.read_sql('SELECT * FROM Transactions', conn)
df4.head()

,id,created_at,succes,amount,currency,acq_tid,card_id,cb
0,1507276519,2017-10-06 09:55:19,1,1300,USD,2DD77FW7-1507276519-100375145-6UDOLBNLXD,1492866731,0
1,1512262637,2017-12-03 01:57:17,1,400,USD,2DD77FW7-1512262637-100375145-5JEOCILQ7T,1492866731,0
2,1509745157,2017-11-03 22:39:17,0,3100,USD,2DD77FW7-1509745157-100375982-BTT12YQNUZ,1492866731,0
3,1507921724,2017-10-13 21:08:44,1,1300,EUR,2DD77FW7-1507921724-100375856-TRGTYMVYHO,1508145460,0
4,1514277559,2017-12-26 09:39:19,0,900,USD,2DD77FW7-1514277559-100632165-2IGY9D935C,1508145460,0


In [12]:
df4.shape

(34887, 8)

Excel file to sql:

In [14]:
df1.to_sql("Report", conn, if_exists="replace")

In [13]:
df5 = pd.read_sql_query("SELECT * FROM Report", conn)
df5.head()

,index,Transaction Datetime,Currnecy,Amount,Masked CCN,Card Brans,ARN
0,0,2017-10-02 01:19:58,USD,70,483316******3047,visa,2DD77FW7-1506899998-100375982-D4QGK72CC0
1,1,2017-10-02 08:21:16,EUR,100,403905******6257,visa,2DD77FW7-1506925276-100375856-3CLLAK8JZ0
2,2,2017-10-02 16:57:29,USD,50,485340******8306,visa,2DD77FW7-1506956249-100632165-A35L6E461Z
3,3,2017-10-03 06:14:16,EUR,240,494172******3012,visa,2DD77FW7-1507004056-100632165-HZCR6EGFSX
4,4,2017-10-03 09:44:27,EUR,120,443047******8271,visa,2DD77FW7-1507016667-100375856-WEYWF1CIRI


We rename columns:

In [14]:
trans = df1.rename(columns={"Transaction Datetime": "created_at", "ARN": "acq_tid", "Currnecy": "currency"})
trans.head()

,created_at,currency,Amount,Masked CCN,Card Brans,acq_tid
0,2017-10-02 01:19:58,USD,70,483316******3047,visa,2DD77FW7-1506899998-100375982-D4QGK72CC0
1,2017-10-02 08:21:16,EUR,100,403905******6257,visa,2DD77FW7-1506925276-100375856-3CLLAK8JZ0
2,2017-10-02 16:57:29,USD,50,485340******8306,visa,2DD77FW7-1506956249-100632165-A35L6E461Z
3,2017-10-03 06:14:16,EUR,240,494172******3012,visa,2DD77FW7-1507004056-100632165-HZCR6EGFSX
4,2017-10-03 09:44:27,EUR,120,443047******8271,visa,2DD77FW7-1507016667-100375856-WEYWF1CIRI


We compare the data from both files and check if they are the same:

In [15]:
df_merged = pd.merge(trans[['created_at', 'Amount', 'acq_tid']], df4[['created_at', 'amount', 'acq_tid']], on='acq_tid')

In [16]:
df_merged.sample(10)

,created_at_x,Amount,acq_tid,created_at_y,amount
185,2017-12-28 08:07:39,200,2DD77FW7-1514444859-100375856-W39GZ5TYEI,2017-12-28 08:07:39,2000
91,2017-11-12 13:07:43,60,2DD77FW7-1510488463-100375856-BVE2Z6DRHP,2017-11-12 13:07:43,600
17,2017-10-08 16:03:27,210,2DD77FW7-1507471407-100375145-4Z5O5MJY6N,2017-10-08 16:03:27,2100
172,2017-12-21 21:12:10,30,2DD77FW7-1513887130-100375145-1IICJT5X58,2017-12-21 21:12:10,300
186,2017-12-28 12:16:27,270,2DD77FW7-1514459787-110034742-L76YZD8CFP,2017-12-28 12:16:27,2700
61,2017-10-26 23:38:23,100,2DD77FW7-1509053903-110034742-L1NXHOG3E2,2017-10-26 23:38:23,1000
121,2017-11-27 23:25:05,230,2DD77FW7-1511821505-100375982-FCFCVUVZD2,2017-11-27 23:25:05,2300
182,2017-12-26 04:28:32,180,2DD77FW7-1514258912-100632165-VQKYSM3AWW,2017-12-26 04:28:32,1800
45,2017-10-20 09:52:19,220,2DD77FW7-1508485939-100375982-TSSSVI47ET,2017-10-20 09:52:19,2200
38,2017-10-15 23:11:31,140,2DD77FW7-1508101891-100375856-XVR7MK4UNY,2017-10-15 23:11:31,1400


We combine the tables:

In [17]:
data = pd.merge(df4, trans[['Masked CCN', 'Amount', 'Card Brans', 'acq_tid']], how='left', on='acq_tid')
data.head()

,id,created_at,succes,amount,currency,acq_tid,card_id,cb,Masked CCN,Amount,Card Brans
0,1507276519,2017-10-06 09:55:19,1,1300,USD,2DD77FW7-1507276519-100375145-6UDOLBNLXD,1492866731,0,NaN,NaN,NaN
1,1512262637,2017-12-03 01:57:17,1,400,USD,2DD77FW7-1512262637-100375145-5JEOCILQ7T,1492866731,0,NaN,NaN,NaN
2,1509745157,2017-11-03 22:39:17,0,3100,USD,2DD77FW7-1509745157-100375982-BTT12YQNUZ,1492866731,0,NaN,NaN,NaN
3,1507921724,2017-10-13 21:08:44,1,1300,EUR,2DD77FW7-1507921724-100375856-TRGTYMVYHO,1508145460,0,NaN,NaN,NaN
4,1514277559,2017-12-26 09:39:19,0,900,USD,2DD77FW7-1514277559-100632165-2IGY9D935C,1508145460,0,NaN,NaN,NaN


In [18]:
data.shape

(34887, 11)

In [20]:
data[data['acq_tid']== '2DD77FW7-1507004056-100632165-HZCR6EGFSX']

,id,created_at,succes,amount,currency,acq_tid,card_id,cb,Masked CCN,Amount,Card Brans
100,1507004056,2017-10-03 06:14:16,1,2400,EUR,2DD77FW7-1507004056-100632165-HZCR6EGFSX,1495394992,0,494172******3012,240.0,visa


In [21]:
data['cb'].unique()

array([0], dtype=int64)

In [30]:
data['Amount'] = data['Amount'].fillna('0')
data['Masked CCN'] = data['Masked CCN'].fillna('0')

In [23]:
data['Amount'] = data['Amount'].astype(str)

We adjust the **cb** value to whether there is a chargeback or not:

In [24]:
for index in data.index:
    if data.loc[index,'Amount'] == '0':
        data.loc[index,'cb'] = 0
    else:
        data.loc[index,'cb'] = 1

In [33]:
data[data['acq_tid']== '2DD77FW7-1507004056-100632165-HZCR6EGFSX']

,id,created_at,succes,amount,currency,acq_tid,card_id,cb,Masked CCN,Amount,Card Brans
100,1507004056,2017-10-03 06:14:16,1,2400,EUR,2DD77FW7-1507004056-100632165-HZCR6EGFSX,1495394992,1,494172******3012,240.0,visa


In [34]:
data[data['acq_tid']== '2DD77FW7-1509745157-100375982-BTT12YQNUZ']

,id,created_at,succes,amount,currency,acq_tid,card_id,cb,Masked CCN,Amount,Card Brans
2,1509745157,2017-11-03 22:39:17,0,3100,USD,2DD77FW7-1509745157-100375982-BTT12YQNUZ,1492866731,0,0,0,NaN


In [28]:
data['cb'].unique()

array([0, 1], dtype=int64)

We rename the Amount column on Amount_cb: 

In [36]:
data = data.rename(columns={"Amount": "Amount_cb"})
data.head()

,id,created_at,succes,amount,currency,acq_tid,card_id,cb,Masked CCN,Amount_cb,Card Brans
0,1507276519,2017-10-06 09:55:19,1,1300,USD,2DD77FW7-1507276519-100375145-6UDOLBNLXD,1492866731,0,0,0,NaN
1,1512262637,2017-12-03 01:57:17,1,400,USD,2DD77FW7-1512262637-100375145-5JEOCILQ7T,1492866731,0,0,0,NaN
2,1509745157,2017-11-03 22:39:17,0,3100,USD,2DD77FW7-1509745157-100375982-BTT12YQNUZ,1492866731,0,0,0,NaN
3,1507921724,2017-10-13 21:08:44,1,1300,EUR,2DD77FW7-1507921724-100375856-TRGTYMVYHO,1508145460,0,0,0,NaN
4,1514277559,2017-12-26 09:39:19,0,900,USD,2DD77FW7-1514277559-100632165-2IGY9D935C,1508145460,0,0,0,NaN


We put the receive file to sql database:

In [37]:
data.to_sql("Transactions_cb", conn, if_exists="replace")

C:\Users\PC\Anaconda3\lib\site-packages\pandas\core\generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [38]:
df6 = pd.read_sql_query("SELECT * FROM Transactions_cb", conn)
df6.head()

,index,id,created_at,succes,amount,currency,acq_tid,card_id,cb,Masked CCN,Amount_cb,Card Brans
0,0,1507276519,2017-10-06 09:55:19,1,1300,USD,2DD77FW7-1507276519-100375145-6UDOLBNLXD,1492866731,0,0,0,None
1,1,1512262637,2017-12-03 01:57:17,1,400,USD,2DD77FW7-1512262637-100375145-5JEOCILQ7T,1492866731,0,0,0,None
2,2,1509745157,2017-11-03 22:39:17,0,3100,USD,2DD77FW7-1509745157-100375982-BTT12YQNUZ,1492866731,0,0,0,None
3,3,1507921724,2017-10-13 21:08:44,1,1300,EUR,2DD77FW7-1507921724-100375856-TRGTYMVYHO,1508145460,0,0,0,None
4,4,1514277559,2017-12-26 09:39:19,0,900,USD,2DD77FW7-1514277559-100632165-2IGY9D935C,1508145460,0,0,0,None
